In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from sdv.tabular import CTGAN
from sdv.evaluation import evaluate
from sdmetrics.reports.single_table import QualityReport
from sdmetrics.reports.utils import get_column_plot
from sdmetrics.single_column import BoundaryAdherence
from sdmetrics.single_column import StatisticSimilarity
import plotly.express as px
import plotly.graph_objects as go

In [8]:
#Selecionando as colunas que serão retiradas do dataset.

df_raw = pd.read_csv('infringement_clean.csv')

df_cleaned = df_raw.drop(columns =  ['provided_mobilephone', 'provided_workphone', 
                                           'provided_homephone', 'provided_email', 'mobilephone_reachable', 
                                           'full_name'])
#Selecionamos 50% dos dados para gerar os dados sintéticos,
#utilizamos a função .sample, da lib pandas, para isso.
#Isso se dá para não deixarmos o processamentos desses dados pelo modelo que irá
#gerar os dados muito pesado.
df_cleaned = df_cleaned.sample(frac=0.15)
df_cleaned.dropna(inplace=True)
df_cleaned

,Unnamed: 0,loan_id,infringed,contract_type,gender,has_own_car,has_own_realty,num_children,annual_income,credit_amount,...,num_req_bureau_qrt,num_req_bureau_year,past_avg_amount_annuity,past_avg_amt_application,past_avg_amt_credit,past_loans_approved,past_loans_refused,past_loans_canceled,past_loans_unused,past_loans_total
121489,121489,240845,0,Revolving loans,M,Y,Y,0,180000.0,270000.0,...,0.0,5.0,15173.5770,129737.000000,129208.000000,3.0,1.0,5.0,0.0,9.0
175591,175591,303471,0,Revolving loans,F,N,Y,0,112500.0,202500.0,...,0.0,6.0,5905.6875,29823.750000,29823.750000,2.0,2.0,0.0,0.0,4.0
174548,174548,302276,0,Cash loans,F,N,Y,2,225000.0,344043.0,...,0.0,1.0,12867.1200,47137.500000,42637.500000,1.0,0.0,0.0,1.0,2.0
237675,237675,375301,0,Cash loans,M,Y,N,0,103500.0,566055.0,...,0.0,1.0,3012.2460,29409.390000,26761.500000,1.0,0.0,4.0,0.0,5.0
241655,241655,379814,0,Cash loans,M,N,Y,0,247500.0,254700.0,...,0.0,3.0,25570.0350,303323.250000,329138.250000,5.0,1.0,0.0,0.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235420,235420,372690,0,Cash loans,F,N,Y,1,135000.0,270126.0,...,0.0,1.0,7116.8625,108528.750000,69419.250000,2.0,0.0,0.0,0.0,2.0
50821,50821,158854,0,Cash loans,M,Y,Y,0,202500.0,1506816.0,...,0.0,0.0,27509.8590,388800.000000,391027.500000,3.0,0.0,2.0,0.0,5.0
277774,277774,421821,0,Cash loans,F,N,N,2,135000.0,1012500.0,...,0.0,4.0,14017.3425,156167.181818,201460.909091,7.0,0.0,4.0,0.0,11.0
97740,97740,213488,0,Cash loans,F,N,N,3,117000.0,127350.0,...,1.0,5.0,7846.5375,54000.000000,59940.000000,2.0,3.0,0.0,0.0,5.0


In [9]:
score = []

for epoch in range(5,30, 5):
    for discriminator in range(1,4):
        model = CTGAN(epochs=epoch, verbose=True, primary_key='loan_id', discriminator_steps=discriminator)
        model.fit(df_cleaned)
        new_data = model.sample(num_rows=500)
        overall_score = evaluate(new_data, df_cleaned)
        score.append([epoch,discriminator,overall_score])